In [2]:
import torch 
from dataset import Pic_to_Pic_dataset
from models import UNET, U2NET
from torch.utils.data import DataLoader
from loss import SSIM_DICE_BCE, DiceScore
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from PIL import Image 
from torchvision.transforms import ToTensor, ToPILImage
import os 
from tqdm import tqdm 
import cv2
from typing import List

torch.set_grad_enabled(False)

In [ ]:
os.system('rm -r plots')

In [ ]:
  

def get_depth_gen(length: int):
    lst = list()
    for i in range(length):
        lst.extend([i]*10)
    for depth in lst:
        yield depth

In [ ]:
def forward_hook(module, input, output): 
    batch_depth = next(depth_gen)
    batch_size = output.shape[0]

    for depth_, op in enumerate(output): 
        depth = str(batch_size*batch_depth+ depth_)
        # print(depth)
        for idx, img in enumerate(op): 
            path = os.path.join('plots', patient_id, depth, module.name)
            os.makedirs(path, exist_ok=True)
            idx = str(len(os.listdir(path)))
            img = (img-img.min())/(img.max()-img.min())
            # print(path)
            ToPILImage()(img).save(path+ '/' + str(idx) + '.png')


In [ ]:
model = UNET().to('cuda:1')
model.assign_names()
ckpt = torch.load('./ckpts/quantum_noise/56/best_unet.pth') 
model.load_state_dict(ckpt['model_state'])
dice_score = DiceScore()
# print(model)


model.ch.register_forward_hook(forward_hook)
model.down1.register_forward_hook(forward_hook)
model.down2.register_forward_hook(forward_hook)
model.down3.register_forward_hook(forward_hook)
model.down4.register_forward_hook(forward_hook)
model.up1.register_forward_hook(forward_hook)
model.up2.register_forward_hook(forward_hook)
model.up3.register_forward_hook(forward_hook)
model.up4.register_forward_hook(forward_hook)
model.out.register_forward_hook(forward_hook)

In [ ]:
whole_df = pd.read_csv('/home/shivac/qml-data/csv_files/val_10_org.csv') 
patient_ids = whole_df['patient_id'].unique()
patient_id = np.random.choice(patient_ids)
global batch_num 
batch_num = 0
img_paths = whole_df[whole_df['patient_id'] == patient_id]['img_path'].values
img_paths = [os.path.join('/home/shivac/qml-data/', img_path) for img_path in img_paths]
batch_img = list() 
for img_path in img_paths: 
    img = Image.open(img_path).convert('L') 
    img = ToTensor()(img)
    batch_img.append(img)
batch_img = torch.stack(batch_img)
batch_size = 10
batch_img = batch_img.split(batch_size)
depth_gen = get_depth_gen(len(img_paths)//batch_size+1)
print(f'{len(batch_img)}')
with torch.no_grad():
    for img in tqdm(batch_img): 
        logits = model(img.to('cuda:1'))

In [11]:
def imgs_to_vid(path): 
    imgs = os.listdir(path)
    imgs = [os.path.join(path, img) for img in imgs if img.endswith('.png')]
    imgs = sorted(imgs, key=lambda x: int(x.split('/')[-1].split('.')[0]))
    if len(imgs) == 0: 
        print('No images found in {}'.format(path))
        return
    img = cv2.imread(imgs[0])
    video = cv2.VideoWriter('{}/vid.mp4'.format(path), cv2.VideoWriter_fourcc(*'mp4v'), 10, (img.shape[1], img.shape[0]))
    
    for img_path in tqdm(imgs): 
        if not img_path.endswith('.png'): 
            continue
        img = cv2.imread(img_path)
        video.write(img)       
    video.release()
    os.system(f'rm {path}/*.png')



In [12]:
plots_path = 'plots_imgs/'
patient_dirs = os.listdir(plots_path) 
for patient_dir in patient_dirs:
    patient_dir_path = os.path.join(plots_path, patient_dir)
    for depth in tqdm(os.listdir(patient_dir_path)): 
        depth_path = os.path.join(patient_dir_path, depth)
        for module in os.listdir(depth_path): 
            module_path = os.path.join(depth_path, module)
            imgs_to_vid(module_path)
           




  0%|          | 0/300 [00:00<?, ?it/s]

No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/down2
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/out
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/down3
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/up3
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/up4
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/up2
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/down1
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/down4
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/ch
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/158/up1
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/81/down2
No images found in plots_imgs/MEDVID0082_M_20211125_175015_0001_IMAGES/81/out
No images found in plots_imgs/MEDVID0082_M_20

100%|██████████| 300/300 [03:55<00:00,  1.28it/s]


In [13]:
whole_df = pd.read_csv('/home/shivac/qml-data/csv_files/val_10_org.csv') 
patient_ids = np.unique(whole_df.patient_id)
model.eval()
for patient_id in patient_ids:
    os.makedirs('plots/{}'.format(patient_id), exist_ok=True)
    df = whole_df[whole_df.patient_id == patient_id].sort_values('idx')
    df.reset_index(inplace=True)
    path_gen = path_gen_fn(['plots/{}/{}/'.format(patient_id, i) for i in range(len(df))])
    for i in tqdm(range(len(df))): 
        os.makedirs('plots/{}/{}'.format(patient_id, i), exist_ok=True)
        img_path = '/home/shivac/qml-data/' + df.loc[i].img_path
        mask_path = '/home/shivac/qml-data/' + df.loc[i].mask_path
        img = Image.open(img_path).convert('L')                         
        mask = Image.open(mask_path) 
        mask = ToTensor()(mask).unsqueeze(0)
        img = ToTensor()(img).unsqueeze(0)
        logits = model(img.cuda())
        dice = round(dice_score(mask.cuda(), logits).item(), 2)

        plt.figure(figsize=(10, 6), facecolor='gray')
        plt.axis('off')
        plt.title('Depth: ' + str(i) + ' dice_score: ' + str(dice))
        plt.subplot(1,3,1)
        plt.title('img')
        plt.axis('off')
        plt.imshow(img[0].permute(1,2,0), cmap='gray')
        plt.subplot(1,3,2)
        plt.title('mask')
        plt.axis('off')
        plt.imshow(mask[0].permute(1,2,0), cmap='gray')
        plt.subplot(1,3,3)
        plt.title('logits')
        plt.axis('off')
        plt.imshow(logits[0].detach().cpu().permute(1,2,0), cmap='gray')
        plt.savefig('plots/{}/{}/res.png'.format(patient_id, i))
        plt.clf() 
        plt.close()
        for path in os.listidir('plots/{}/{}'.format(patient_id, i)):
            imgs_to_vid(os.path.join(path, 'ch/'))
    break
        
        


NameError: name 'model' is not defined

In [ ]:
patient_id